In [1]:
import re,string,nltk
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize 
import numpy as np
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_sm')
import string
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer  # https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.tree import DecisionTreeClassifier

from collections import Counter
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
import math
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [37]:
data = pd.read_csv('/kaggle/input/d/ritikabhardwaj2801/mediaeval2015/mediaeval-2015-trainingset.txt', sep = '\t')
#data.head()
data.columns = ["tweetId", "tweetText", "userId", "imageId(s)","username","timestamp","label"]
data.dropna(inplace=True)
data=data.drop_duplicates(subset=['tweetText'])
def labels_to_numbers(label):
    if label[0] == 'r':
        return 0
    else:
        return 1
data['label'] = data['label'].apply(labels_to_numbers)

In [39]:
def get_number_of_words(text):
    x=text.split(" ")
    count=len(x)
    return count

In [40]:
data['newFeature'] = data['tweetText'].apply(get_number_of_words)

In [41]:
def process_text(text): #should probably get rid of all the remaining hashtags ..or not 
    text = text.translate(str.maketrans('', '', string.punctuation)) #getting rid of the punctuation
    text_tokens = nlp(text)
    text_tokens = [token.lemma_ for token in text_tokens if not token.is_stop] #getting rid of hashtags and tags , and token != "" and token[0] not in ['@', '#'])
    return " ".join(text_tokens)

In [42]:
def preprocess_text(text):
    text = text.lower()
    new_text = []
    for t in text.split(" "):
        t = '' if (t.startswith('http')) else t #t = '' if (t.startswith('@') or t.startswith('#') or  t.startswith('http')) else t
        if t != '':
            new_text.append(t)
    return " ".join(new_text)

data['tweetText'] = data['tweetText'].apply(preprocess_text)
data=data.dropna()

In [43]:
data['tweetTextTokens'] = data['tweetText'].apply(process_text)
data=data.dropna()
data = data.drop(columns=['timestamp', 'username', 'userId', 'tweetText'])

In [9]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stopwords.words('english')

def stopword_removal(my_txt):
    filtered_list = []
    stop_words = nltk.corpus.stopwords.words('english')
# Tokenize the sentence
    words = word_tokenize(my_txt)
    for w in words:
        if w.lower() not in stop_words:
            filtered_list.append(w)
    filtered_list
    my_clean_txt = " ".join(filtered_list)
    return my_clean_txt

data['tweetTextTokens'] = data['tweetTextTokens'].apply(stopword_removal)
data.head()

,tweetId,imageId(s),label,newFeature,tweetTextTokens
0,263046056240115712,sandyA_fake_46,1,22,¿ se acuerdan de la película `` el día después...
1,262995061304852481,sandyA_fake_09,1,21,milenagimon miren sandy en ny tremenda imagen ...
2,262979898002534400,sandyA_fake_09,1,19,buena la foto del huracán sandy recuerda la pe...
3,262996108400271360,sandyA_fake_29,1,5,scary shit hurricane ny
4,263018881839411200,sandyA_fake_15,1,12,fave place world nyc hurricane sandy statueofl...


Putting the test data in an usable format

In [44]:
test_data = pd.read_csv('/kaggle/input/d/ritikabhardwaj2801/mediaeval2015/mediaeval-2015-testset.txt', sep = '\t')
test_data.head()

,tweetId,tweetText,userId,imageId(s),username,timestamp,label
0,578854927457349632,kereeen RT @Shyman33: Eclipse from ISS.... htt...,70824972,eclipse_01,peay_s,Fri Mar 20 09:45:43 +0000 2015,fake
1,578874632670953472,Absolutely beautiful! RT @Shyman33: Eclipse fr...,344707006,eclipse_01,JaredUcanChange,Fri Mar 20 11:04:02 +0000 2015,fake
2,578891261353984000,“@Shyman33: Eclipse from ISS.... http://t.co/C...,224839607,eclipse_01,tpjp1231,Fri Mar 20 12:10:06 +0000 2015,fake
3,578846612312748032,Eclipse from ISS.... http://t.co/En87OtvsU6,134543073,eclipse_01,Shyman33,Fri Mar 20 09:12:41 +0000 2015,fake
4,578975333841551360,@ebonfigli: Éclipse vue de l'ISS... Autre chos...,1150728872,eclipse_01,Epimethee_,Fri Mar 20 17:44:11 +0000 2015,fake


In [45]:
test_data['newFeature'] = test_data['tweetText'].apply(get_number_of_words)
test_data['tweetText'] = test_data['tweetText'].apply(preprocess_text)
test_data['tweetText'] = test_data['tweetText'].apply(stopword_removal)
test_data=test_data.dropna()
test_data['tweetTextTokens'] = test_data['tweetText'].apply(process_text)
test_data=test_data.dropna()
test_data['label'] = test_data['label'].apply(labels_to_numbers)
test_target = test_data['label']
test_data = test_data.drop(columns=['timestamp', 'username', 'userId', 'tweetText','label'])

Evaluation

In [46]:
# train_data = data['tweetTextTokens']
train_target = data['label']
train_data = data.drop(columns=['label'])

Vectorizing train and test set

In [47]:
tfid_vectorizer = TfidfVectorizer()
vectorized_train = tfid_vectorizer.fit_transform(train_data['tweetTextTokens'])
vectorized_test = tfid_vectorizer.transform(test_data['tweetTextTokens'])

In [48]:
count_vectorizer = CountVectorizer()
vectorized_train2 = count_vectorizer.fit_transform(train_data['tweetTextTokens'])
vectorized_test2 = count_vectorizer.transform(test_data['tweetTextTokens'])

Decision tree classifier:

In [49]:
dtree_clf2 = DecisionTreeClassifier(random_state=42)
dtree_clf2.fit(vectorized_train2, train_target)
dtree_pred2 = dtree_clf2.predict(vectorized_test2)
print (f1_score(test_target, dtree_pred2, average='micro'))
print(classification_report(test_target, dtree_pred2))

0.8324900133155793
              precision    recall  f1-score   support

           0       0.68      0.91      0.78      1209
           1       0.95      0.80      0.87      2546

    accuracy                           0.83      3755
   macro avg       0.81      0.85      0.82      3755
weighted avg       0.86      0.83      0.84      3755



In [62]:
dtree_clf2

DecisionTreeClassifier(random_state=42)

In [50]:
from sklearn.linear_model import LogisticRegression
log_clf = LogisticRegression(random_state=42,solver='liblinear',C= 100)
log_clf.fit(vectorized_train2, train_target)
log_pred = log_clf.predict(vectorized_test2)
print (f1_score(test_target, log_pred, average='micro'))
print(classification_report(test_target, log_pred))


0.8090545938748336
              precision    recall  f1-score   support

           0       0.67      0.80      0.73      1209
           1       0.90      0.81      0.85      2546

    accuracy                           0.81      3755
   macro avg       0.78      0.81      0.79      3755
weighted avg       0.82      0.81      0.81      3755



In [51]:
nb_clf = MultinomialNB()
nb_clf.fit(vectorized_train, train_target)
nb_pred = nb_clf.predict(vectorized_test)
print (f1_score(test_target, nb_pred, average='micro'))
print(classification_report(test_target, nb_pred))

0.6239680426098535
              precision    recall  f1-score   support

           0       0.45      0.75      0.56      1209
           1       0.83      0.56      0.67      2546

    accuracy                           0.62      3755
   macro avg       0.64      0.66      0.62      3755
weighted avg       0.70      0.62      0.64      3755



In [58]:
#Import svm model
from sklearn import svm

clf = svm.SVC(kernel='rbf',C=100) # Linear Kernel

clf.fit(vectorized_train, train_target)

y_pred = clf.predict(vectorized_test)

print (f1_score(test_target, y_pred, average='micro'))
print(classification_report(test_target,y_pred))


0.8018641810918775
              precision    recall  f1-score   support

           0       0.68      0.74      0.71      1209
           1       0.87      0.83      0.85      2546

    accuracy                           0.80      3755
   macro avg       0.77      0.79      0.78      3755
weighted avg       0.81      0.80      0.80      3755



In [ ]:
#Import svm model
from sklearn import svm

clf = svm.SVC(kernel='rbf',C=100) # Linear Kernel

clf.fit(vectorized_train, train_target)

y_pred = clf.predict(vectorized_test)

print (f1_score(test_target, y_pred, average='micro'))
print(classification_report(test_target,y_pred))


Random Forrest Classifier

In [64]:
from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(random_state=42)
forest_clf.fit(vectorized_train, train_target)
forest_pred = forest_clf.predict(vectorized_test)
print (f1_score(test_target, forest_pred, average='micro'))
print(classification_report(test_target, forest_pred))

0.43315126489350836
              precision    recall  f1-score   support

           0       0.35      0.85      0.49      1209
           1       0.77      0.25      0.37      2546

    accuracy                           0.44      3755
   macro avg       0.56      0.55      0.43      3755
weighted avg       0.64      0.44      0.41      3755



In [53]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier()
classifier.fit(vectorized_train, train_target)

y_pred_final = classifier.predict(vectorized_test)
print (f1_score(test_target, y_pred_final, average='micro'))
print(classification_report(test_target, y_pred_final ))


0.7723035952063914
              precision    recall  f1-score   support

           0       0.61      0.83      0.70      1209
           1       0.90      0.75      0.82      2546

    accuracy                           0.77      3755
   macro avg       0.75      0.79      0.76      3755
weighted avg       0.81      0.77      0.78      3755



In [63]:
# We'll use 100 weak learners to build a strong learner
from sklearn.ensemble import AdaBoostClassifier
classifier = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(),n_estimators=100)
classifier.fit(vectorized_train2, train_target)
y_pred = classifier.predict(vectorized_test2)

print (f1_score(test_target, y_pred, average='micro'))
print(classification_report(test_target, y_pred ))

0.836750998668442
              precision    recall  f1-score   support

           0       0.68      0.92      0.78      1209
           1       0.96      0.80      0.87      2546

    accuracy                           0.84      3755
   macro avg       0.82      0.86      0.83      3755
weighted avg       0.87      0.84      0.84      3755

